# **Projekt: Fehlererkennung in Getrieben**
## Netztyp 2: FFT-Net

#### **Vorbereitung**

In [ ]:
# import libraries
from uuid import uuid4
import os
from pathlib import Path

# module imports from /src
if Path.cwd().stem == "notebooks":
    os.chdir( Path.cwd().parent)

import src.data_loader as dl
import src.visualization as vis
import src.net_models as net
import src.input_preparation as ip

In [ ]:
# set the source directory for the preprocessed data to use
SOURCE: str = "f2fcf2aa-bd96-4d89-8bb5-4a0a1bc11b1b"

# setup a single index for control purposes
CONTROL_INDEX: int = 0

# generate a unique identifier for this run
UUID: str = str(uuid4())
print(f"Current UUID:\t{UUID}")

In [ ]:
# setup system and check the number of cpu cores and gpus available
net.system_setup()

#### **Laden der Datensätze**

In [ ]:
# load data from folder and split in training and evaluation data
data_path = Path().cwd() / "data" / "processed" / SOURCE
development_data, evaluation_data = dl.load_all_datasets(data_path)

# define file path for metrics and confusion matrix
figure_path = Path().cwd() / "results" / "metrics" / ("b_" + UUID)

In [ ]:
# visualize one random dataset for data validation
_ = vis.plot_column_data(development_data[CONTROL_INDEX],
                             development_data[CONTROL_INDEX].columns)

#### **Das Neuronale Netz**

In [ ]:
full_development_dataset = ip.concatenate_datasets(development_data, use_binary_labeling=True)

training_samples_dict = ip.split_data(
    dataframe=full_development_dataset,
    data_columns=[
        "vibration_1_magnitude",
        # "vibration_2_magnitude",
        # "vibration_3_magnitude",
    ],
)

In [ ]:
ip.check_data(training_samples_dict)

print(training_samples_dict["samples"].shape)

In [ ]:
# model creation parameters
N_HIDDEN_LAYERS: int = 2
L2: float = 2e-2 # 0.02
DROPOUT: float = 0.2
NEGATIVE_SLOPE: float = 0.01 #0.3

# model compilation parameters
LEARNING_RATE: float = 1e-6

# model training parameters
BATCH_SIZE: int = 128
EPOCHS: int = 200
VALIDATION_SPLIT: float = 0.1

# manual changes to class weights
training_samples_dict["class_weights"] = {
    0: 1.0,  # none (Klasse 0) wird weniger stark gewichtet
    1: 1.0,  # some (Klasse 1) wird stärker gewichtet
}

In [ ]:
model = net.binary_classifier.build_model(n_hidden_layers=N_HIDDEN_LAYERS,
                                   training_samples_dict=training_samples_dict,
                                   l2=L2,
                                   dropout=DROPOUT,
                                   negative_slope=NEGATIVE_SLOPE)

In [ ]:
net.binary_classifier.compile(model=model, learning_rate=LEARNING_RATE)

history = net.train(
    model=model,
    samples_dict=training_samples_dict,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    use_early_stopping=True,
)

#### **Bewertung des Neuronalen Netzes**

In [ ]:
# visualize training metrics
figure_dicts = vis.plot_history(
    history,
    metrics=["accuracy", "f1_score", "loss", "precision", "recall"],
    separated=True,
)

for figure_dict in figure_dicts:
    dl.save_figure(
        parent_path=figure_path,
        figure_dict=figure_dict,
        format="png",
        dpi=300,
    )

In [ ]:
full_evaluation_dataset = ip.concatenate_datasets(
    evaluation_data, use_binary_labeling=True
)

test_samples_dict = ip.split_data(
    dataframe=full_evaluation_dataset,
    data_columns=[
        "vibration_1_magnitude",
        # "vibration_2_magnitude",
        # "vibration_3_magnitude",
    ],
)

In [ ]:
ip.check_data(test_samples_dict)

In [ ]:
# evaluate model
evaluation = net.evaluate(model=model, test_samples_dict=test_samples_dict, batch_size=BATCH_SIZE)
display(evaluation)

### Confusion Matrix
Die Confusion Matrix zeigt, wie gut dein Modell zwischen den verschiedenen Klassen unterscheidet. Jede Zelle stellt die Anzahl der Samples dar, die einer bestimmten Klasse zugeordnet wurden (wahre Labels) und wie das Modell diese Klasse vorhergesagt hat (vorhergesagte Labels).



In [ ]:
# predictions by model
true_labels, predicted_labels = net.predict(
    model=model, test_samples_dict=test_samples_dict
)

print(type(test_samples_dict["encoder"].classes_))

# plot confusion matrix
figure_dict = vis.plot_confusion_matrix(
    true_labels=true_labels,
    predicted_labels=predicted_labels,
    class_names=test_samples_dict["encoder"].classes_.tolist(),
)


dl.save_figure(
    parent_path=figure_path,
    figure_dict=figure_dict,
    format="png",
    dpi=300,
)

dl.save_model_hyperparameters(
    parent_path=figure_path,
    model=model,
    learning_rate = LEARNING_RATE,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=VALIDATION_SPLIT,
)